<a href="https://colab.research.google.com/github/j00lee/SignLingo/blob/main/Selection_of_Vocabulary_%2B_Trimming_the_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step One: Verify that the selected words list contains enough words to create full sentences

In [ ]:
# Mount Google Drive first
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from collections import defaultdict
import string

# Paths to your files
words_file = '/content/drive/MyDrive/ASL Project/gloss_freq_exploration/words_list.txt'
csv_file = '/content/drive/MyDrive/ASL Project/gloss_freq_exploration/slt_data_clean.csv'

# Load the word list (normalize to uppercase, exclude non-word lines)
with open(words_file, 'r') as f:
    allowed_words = set(
        line.strip().upper() for line in f if line.strip().isalpha()
    )

# Load the SLT data CSV
df = pd.read_csv(csv_file)

# Define punctuation set
punctuation_set = set(string.punctuation)

# Dictionary to store missing gloss counts (for rows with <= 2 missing)
missing_gloss_counts = defaultdict(int)

# Function to check gloss validity and track missing
def check_gloss(gloss):
    tokens = gloss.strip().split()
    missing = [
        tok.upper() for tok in tokens
        if tok not in punctuation_set and tok.upper() not in allowed_words
    ]

    if len(missing) <= 2:
        for gloss in missing:
            missing_gloss_counts[gloss] += 1

    return len(missing) == 0

# Apply the function and track valid inputs
df['valid'] = df['input'].apply(check_gloss)

# Summary
num_valid = df['valid'].sum()
total = len(df)

print(f"Valid gloss inputs: {num_valid} out of {total}")
print(f"Percentage: {num_valid / total * 100:.2f}%\n")

# Print missing gloss summary (for inputs with ≤ 2 missing glosses)
print("Missing glosses from inputs with ≤ 2 missing glosses (excluding punctuation):")
for gloss, count in sorted(missing_gloss_counts.items(), key=lambda x: -x[1]):
    print(f"{gloss}: {count}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Valid gloss inputs: 78 out of 7135
Percentage: 1.09%

Missing glosses from inputs with ≤ 2 missing glosses (excluding punctuation):
I: 311
WHERE: 74
NAME: 57
SHE: 55
DEAF: 47
SIGN: 42
FINISH: 42
BOOK: 32
LIVE: 28
HOW-MANY: 28
SISTER: 27
OLD: 27
YESTERDAY: 25
HEAR: 25
EAT: 23
COLOR: 23
FAMILY: 22
HERE: 22
FEEL: 21
MAN: 21
FAVORITE: 20
HOME: 20
PLEASE: 18
TEACH+AM: 17
HOUSE: 17
IT: 17
WHAT-KIND: 16
TEACHER: 16
ESCAPE: 15
MOVIE: 15
BREAK: 15
SCHOOL: 15
NEED: 15
TOMORROW: 15
LOVE: 15
LEARN: 14
PLAY: 14
HAPPY: 13
MONEY: 13
YES: 13
WOMAN: 13
BIRTHDAY: 13
CHILDREN: 13
MUST: 13
TODAY: 13
BEFORE: 13
ASL: 12
FOOD: 12
PAPER: 12
HER: 12
FATHER: 11
NONE: 11
THIS: 11
LOSE: 11
HORSE: 10
TABLE: 10
WIFE: 10
UNDERSTAND: 10
NOTHING: 10
BOY: 10
NICE: 10
DO: 10
T-V: 10
AUNT: 10
SICK: 10
DAD: 10
PRETTY: 9
EXERCISE: 9
WHY: 9
WEATHER: 9
LEARN+AM: 9
DOCTOR: 9
WRITE: 9
NOT-YET: 8
MEET

## See if the top 20 gloss in the global dictionary is present in the video dataset

In [ ]:
# Define the list of glosses to check
glosses_to_check = [
    "I", "WHERE", "NAME", "SHE", "DEAF", "SIGN", "FINISH", "BOOK", "LIVE", "HOW-MANY",
    "SISTER", "OLD", "YESTERDAY", "HEAR", "EAT", "COLOR", "FAMILY", "HERE", "FEEL", "MAN"
]

# Path to the common_glosses.txt file
common_glosses_path = "drive/MyDrive/ASL Project/gloss_freq_exploration/common_glosses.txt"

# Load common glosses from file
with open(common_glosses_path, "r") as f:
    common_glosses = set(line.strip().upper() for line in f if line.strip())


# Print only the glosses that are found, one per line
for gloss in glosses_to_check:
    if gloss in common_glosses:
        print(gloss)


WHERE
NAME
DEAF
SIGN
FINISH
BOOK
LIVE
SISTER
OLD
YESTERDAY
HEAR
EAT
COLOR
FAMILY
HERE
FEEL
MAN


## See how many inputs in Achraf's dataset consist of gloss from our gloss list

In [ ]:
# Mount Google Drive first
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import string

# Paths to your files
words_file = '/content/drive/MyDrive/ASL Project/gloss_freq_exploration/words_list.txt'
csv_file = '/content/drive/MyDrive/ASL Project/gloss_freq_exploration/slt_data_clean.csv'

# Load the word list (normalize to uppercase, exclude non-word lines like section headers)
with open(words_file, 'r') as f:
    allowed_words = set(
        line.strip().upper() for line in f if line.strip().isalpha()
    )

# Load the SLT data CSV
df = pd.read_csv(csv_file)

# Define a function to check if all gloss tokens are allowed (ignoring punctuation)
def is_valid_gloss(gloss):
    tokens = gloss.strip().split()
    return all(
        token.upper() in allowed_words or all(c in string.punctuation for c in token)
        for token in tokens
    )

# Apply the function to each row
df['valid'] = df['input'].apply(is_valid_gloss)

# Count valid and total examples
num_valid = df['valid'].sum()
total = len(df)

# Print the results
print(f"Valid gloss inputs: {num_valid} out of {total}")
print(f"Percentage: {num_valid / total * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Valid gloss inputs: 79 out of 7135
Percentage: 1.11%


# Ok finally, we filter the BIG dataset for only the videos of these 128 gloss

In [ ]:
import pandas as pd
import re

# Path to gloss list
gloss_list_path = '/content/drive/MyDrive/ASL Project/gloss_freq_exploration/words_list.txt'

# Path to original splits
split_folder = '/content/drive/MyDrive/ASL Project/splits'

# Output filenames
splits = ['train', 'val', 'test']

# Load allowed glosses (uppercase)
with open(gloss_list_path, 'r') as f:
    allowed_glosses = set(line.strip().upper() for line in f if line.strip().isalpha())

# Function to clean gloss for comparison (but not for saving)
def clean_gloss(gloss):
    gloss = gloss.strip().upper()
    return re.sub(r'\s*\d+$', '', gloss)

# Process each split
for split in splits:
    input_csv = f'{split_folder}/{split}.csv'
    output_csv = f'{split_folder}/{split}_filtered.csv'

    # Read CSV
    df = pd.read_csv(input_csv)

    # Keep a cleaned version only for filtering
    df['Gloss_clean'] = df['Gloss'].apply(clean_gloss)

    # Filter using the cleaned gloss
    df_filtered = df[df['Gloss_clean'].isin(allowed_glosses)]

    # Drop helper column and save
    df_filtered.drop(columns=['Gloss_clean'], inplace=True)
    df_filtered.to_csv(output_csv, index=False)

    print(f"{split}: {len(df_filtered)} rows retained out of {len(df)}")


<ipython-input-12-91fb276d4fe7>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['Gloss_clean'], inplace=True)


train: 2031 rows retained out of 40154


<ipython-input-12-91fb276d4fe7>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['Gloss_clean'], inplace=True)


val: 504 rows retained out of 10304
test: 1692 rows retained out of 32941


<ipython-input-12-91fb276d4fe7>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['Gloss_clean'], inplace=True)


## Further trim the test set (to 500 videos)

In [ ]:
import pandas as pd
import random
from collections import defaultdict, Counter

# Set random seed for reproducibility
random.seed(42)

# Paths
base_path = '/content/drive/MyDrive/ASL Project/splits'
words_list_path = '/content/drive/MyDrive/ASL Project/gloss_freq_exploration/words_list.txt'
train_csv_path = f'{base_path}/train_filtered.csv'
test_csv_path = f'{base_path}/test_filtered.csv'
output_csv_path = f'{base_path}/test_final.csv'

# Load gloss list
with open(words_list_path, 'r') as f:
    allowed_glosses = set(line.strip().upper() for line in f if line.strip().isalpha())

# Load filtered CSVs
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

# STEP 1: Create gloss → [videos] dictionary from test set
test_videos_by_gloss = defaultdict(list)
for _, row in df_test.iterrows():
    gloss = row['Gloss'].strip().upper()
    if gloss in allowed_glosses:
        test_videos_by_gloss[gloss].append(row)

# STEP 2: Get frequency distribution from train set
train_gloss_counts = Counter(
    row['Gloss'].strip().upper()
    for _, row in df_train.iterrows()
    if row['Gloss'].strip().upper() in allowed_glosses
)

total_train = sum(train_gloss_counts.values())

# STEP 3: Compute number of videos per gloss (proportional to train dist, sum to 500)
desired_total = 500
target_count_per_gloss = {
    gloss: round((count / total_train) * desired_total)
    for gloss, count in train_gloss_counts.items()
}

# STEP 4: Select randomly from test videos per gloss
final_test_rows = []
for gloss, target_count in target_count_per_gloss.items():
    available = test_videos_by_gloss.get(gloss, [])
    selected = random.sample(available, min(target_count, len(available)))
    final_test_rows.extend(selected)

# Trim or pad to exactly 500 if necessary
if len(final_test_rows) > 500:
    final_test_rows = random.sample(final_test_rows, 500)
elif len(final_test_rows) < 500:
    needed = 500 - len(final_test_rows)
    # Gather more random test videos from glosses that still have unused entries
    extras = []
    for gloss in allowed_glosses:
        remaining = [row for row in test_videos_by_gloss.get(gloss, []) if row not in final_test_rows]
        extras.extend(remaining)
    final_test_rows.extend(random.sample(extras, min(needed, len(extras))))

# Save final test set
df_final = pd.DataFrame(final_test_rows)
df_final.to_csv(output_csv_path, index=False)

print(f"Saved test_final.csv with {len(df_final)} videos.")


Saved test_final.csv with 500 videos.


# WE TRIM THE VIDEOS AND UPLOAD TO DROPBOX

In [ ]:
# STEP 1: Mount Google Drive to access your filtered CSVs and raw videos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# STEP 2: Install Dropbox SDK
!pip install dropbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 12.8 MB/s eta 0:00:00


In [ ]:
!pip install numpy==2.0.0 --force-reinstall
!pip install mediapipe --no-deps

  Using cached numpy-2.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires sounddevice>=0.4.4, which is not installed.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.0.0 which is incompatible.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 5.29.4 which is incompatible.


In [ ]:
# STEP 3: Set up MediaPipe + Dropbox + Paths
import os
import cv2
import dropbox
import pandas as pd
import mediapipe as mp
from tqdm import tqdm
from pathlib import Path

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Set paths
SPLIT_DIR = '/content/drive/MyDrive/ASL Project/splits'
VIDEO_DIR = '/content/drive/MyDrive/videos'
TEMP_OUTPUT = '/tmp/trimmed_videos'
SPLITS = ['train', 'val', 'test']
Path(TEMP_OUTPUT).mkdir(exist_ok=True)

# Set up Dropbox connection
DROPBOX_ACCESS_TOKEN = "sl.u.AFu-aIMez3sMU3YziUtieeopVKfJ5c1_4TTZkq8wlu8CSFatc6QEf5tmV5thi63b7qONXfcCKfE51k6vi1y88IaEwbOpFRtOAmR7soByVQw9_-L7MePI4332GOEF0nAT6a2R1oYnKBp78PsjwBoy9hhTPDnuQNPEH2Paud4PrddNuax8h8dJJ6ONre3TE1g3mIqyFn-iIwvzg9DSUS6IgwPRpsJDo0BoaE-Ej5Q6Syj0Gkh1N8EVxVYlKkaHYHSwl3kQtqZb_klD25NwmZSJotudn5BDcE5ouo9qs-Sl4-jSXBaW_dG0ursZ83DdmsA9tLxCMS9d_4sYJ7moS0WDEDHzOkgOkB0fxOfeusiMv2l9MaMmyaT_TOzbs8XSK8Jdrkqd6NEo4Tr24DiAK06MSUwmclBxXNygmCBVffz5rTPurkLWVpDyYBVm_oc5I1FdNc4md-RR463ysPjVxWwpmGe7lhDxPuCEJT-UjGpfQi8zdXO6z4YL1iQwpOsH52khcFQnGKDTHbLHM0LNDUIOZjd9IPYyiC4YuGyM7TKYO5TxIrVpKJdxz9WD-MYOF7K_q8jp3-W7V3xy22iOLgLbBSgJ0g6APRYURktK_p2Z-YJsM-yFGPE11_P-nnvmmo8Xxmvp0VoRy3_npv4Qn-b4DIyQJ6UeUddEZqeZdR4hh5FQeT6DoxSJRo4o-xDV4Q20hAzh7ex1In50v6dZDbM6jY0GV8DMIfrL4xZfO94GA_8fLCJHZObnar-WZQTlp-HZWsQDyp1rvL0FGFbHDc9Ru78yGWArLsfXBCVWKqunfdrmfg0UNGOsVaHYuDa6Nd_s-JlQ9VBAPwIHzVkOJ_jzvw6Q-UG-9w7DV-F5ks8xYk9-pgCsr5N1x8lB2ZxEaEFmD_4SXEVdNSe8Hf13fC6Q8CSqL7xuBXEfpxa-Vw9vDALJROKRgaKvDUdNSY-JgaxnHM0NcgzjNIzHUh4YI42zAbg0YAbDk2NB2JkpngbIg7ShsNcBOWxJPFwzD5mKFdaKLfnpT4LhF9YQhNIEGwdvjZubh6aAWyPFB06jMVhB29TyqscVrcgcSIK0ro05mUMMxbSlFfSDP5xXykNTwE2JNP2wL0uCJo-rYlBKBFQYqaddk1p95fJfZtvmzBzFb9xR9sKV_oz4qPbqgoZhFkK8wCyHWlght4BgWM6qGRWhbyTit7trjcLdlJ0x18rnLGxHyg2YdwgxeIzu2sYqY_4QscUNqoJh3IYHRBY5X0Aq-n1A3EkblTou8H9UZQUQERjG_gM5hbYQrOFlvSrSSD4hJ7FsKtScgraH1Q88T3opxRO0AYVQrKkohtmG-bDaDYRHUVIJpwBpAg_pe4ZLyZbZ1jAiCd8W2zgOC_Qzv5KZntB0h54JQPxBOwmDlJt3XWuxHfeDe3xolRMGL4K4C3BB5MI4JrEqorNxv4TzODa5RN2k1g"
dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
dbx.users_get_current_account()  # Check connection


FullAccount(account_id='dbid:AADBKJnEX5sKMQHomLV8zHHYAisHwN-IlcY', account_type=AccountType('basic', None), country='US', disabled=False, email='bruhhletmecompute@gmail.com', email_verified=True, is_paired=False, locale='en', name=Name(abbreviated_name='JL', display_name='Joohye Lee', familiar_name='Joohye', given_name='Joohye', surname='Lee'), profile_photo_url=NOT_SET, referral_link='https://www.dropbox.com/referrals/AADUYicEqNhK6gVldDgtTo64DMGQZC6RhGM?src=app9-4391555', root_info=UserRootInfo(home_namespace_id='12351690675', root_namespace_id='12351690675'), team=NOT_SET, team_member_id=NOT_SET)

In [ ]:
# STEP 4: Define helper functions
def is_active(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)
    return result.multi_hand_landmarks is not None

def trim_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    active_flags = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        active_flags.append(is_active(frame))

    cap.release()

    active_indices = [i for i, active in enumerate(active_flags) if active]
    if not active_indices:
        print(f"No signing detected in: {video_path}")
        return False

    start = max(min(active_indices) - 5, 0)
    end = min(max(active_indices) + 5, len(frames) - 1)

    height, width, _ = frames[0].shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (width, height))
    for i in range(start, end + 1):
        out.write(frames[i])
    out.release()
    return True

def upload_to_dropbox(local_path, dropbox_path):
    with open(local_path, "rb") as f:
        dbx.files_upload(f.read(), dropbox_path, mode=dropbox.files.WriteMode.overwrite)

def file_exists_in_dropbox(path):
    try:
        dbx.files_get_metadata(path)
        return True
    except dropbox.exceptions.ApiError as e:
        if isinstance(e.error, dropbox.files.GetMetadataError) and e.error.is_path() and e.error.get_path().is_not_found():
            return False
        else:
            raise e


## Test the trimming on 10 videos first

In [ ]:
# STEP 5: Process and upload each split, tracking missing videos
for split in SPLITS:
    print(f"\nProcessing {split} split...")
    csv_path = os.path.join(SPLIT_DIR, f"{split}_filtered.csv")
    df = pd.read_csv(csv_path).sample(n=10, random_state=42)  # Random sample of 10 videos

    missing_count = 0
    skipped_empty_count = 0
    success_count = 0

    for _, row in tqdm(df.iterrows(), total=len(df)):
        video_filename = row['Video file']
        input_path = os.path.join(VIDEO_DIR, video_filename)
        local_output_path = os.path.join(TEMP_OUTPUT, video_filename)
        dropbox_output_path = f"/ASL_trimmed/{split}/{video_filename}"

        if os.path.exists(input_path):
            success = trim_video(input_path, local_output_path)
            if success:
                upload_to_dropbox(local_output_path, dropbox_output_path)
                success_count += 1
            else:
                skipped_empty_count += 1
        else:
            missing_count += 1

    print(f"\nSplit: {split}")
    print(f"  ✅ Trimmed and uploaded: {success_count}")
    print(f"  ⚠️ Skipped (no active signing): {skipped_empty_count}")
    print(f"  ❌ Missing videos: {missing_count}")



Processing train split...


100%|██████████| 10/10 [01:13<00:00,  7.32s/it]



Split: train
  ✅ Trimmed and uploaded: 10
  ⚠️ Skipped (no active signing): 0
  ❌ Missing videos: 0

Processing val split...


100%|██████████| 10/10 [00:47<00:00,  4.73s/it]



Split: val
  ✅ Trimmed and uploaded: 10
  ⚠️ Skipped (no active signing): 0
  ❌ Missing videos: 0

Processing test split...


100%|██████████| 10/10 [00:52<00:00,  5.21s/it]


Split: test
  ✅ Trimmed and uploaded: 10
  ⚠️ Skipped (no active signing): 0
  ❌ Missing videos: 0


## Trimming the full train, val, and test sets

In [ ]:
# STEP 5: Process and upload each split, tracking missing videos
for split in SPLITS:
    print(f"\nProcessing {split} split...")
    csv_path = os.path.join(SPLIT_DIR, f"{split}_filtered.csv")
    df = pd.read_csv(csv_path)

    missing_count = 0
    skipped_empty_count = 0
    success_count = 0

    for _, row in tqdm(df.iterrows(), total=len(df)):
        video_filename = row['Video file']
        input_path = os.path.join(VIDEO_DIR, video_filename)
        local_output_path = os.path.join(TEMP_OUTPUT, video_filename)
        dropbox_output_path = f"/ASL_trimmed/{split}/{video_filename}"

        if os.path.exists(input_path):
            success = trim_video(input_path, local_output_path)
            if success:
                upload_to_dropbox(local_output_path, dropbox_output_path)
                success_count += 1
            else:
                skipped_empty_count += 1
        else:
            missing_count += 1

    print(f"\nSplit: {split}")
    print(f"  ✅ Trimmed and uploaded: {success_count}")
    print(f"  ⚠️ Skipped (no active signing): {skipped_empty_count}")
    print(f"  ❌ Missing videos: {missing_count}")




Processing train split...


 40%|███▉      | 806/2031 [1:05:31<1:17:43,  3.81s/it]

No signing detected in: /content/drive/MyDrive/videos/12041907153156517-DOG 3.mp4


 41%|████      | 825/2031 [1:06:51<1:12:58,  3.63s/it]

No signing detected in: /content/drive/MyDrive/videos/22877680820569246-AND.mp4


...


Split: train
  ✅ Trimmed and uploaded: 2029
  ⚠️ Skipped (no active signing): 2
  ❌ Missing videos: 0

Processing val split...


100%|██████████| 504/504 [43:51<00:00,  5.22s/it]



Split: val
  ✅ Trimmed and uploaded: 504
  ⚠️ Skipped (no active signing): 0
  ❌ Missing videos: 0

Processing test split...


  2%|▏         | 9/500 [00:42<38:12,  4.67s/it]


AuthError: AuthError('6a04e147d35f4de68449b68ad950a757', AuthError('expired_access_token', None))

In [ ]:
for split in ['test']:
    print(f"\nProcessing {split} split...")
    csv_path = os.path.join(SPLIT_DIR, f"{split}_filtered.csv")
    df = pd.read_csv(csv_path)  # remove `.sample()` for full test set

    missing_count = 0
    skipped_empty_count = 0
    success_count = 0

    for _, row in tqdm(df.iterrows(), total=len(df)):
        video_filename = row['Video file']
        input_path = os.path.join(VIDEO_DIR, video_filename)
        local_output_path = os.path.join(TEMP_OUTPUT, video_filename)
        dropbox_output_path = f"/ASL_trimmed/{split}/{video_filename}"

        if os.path.exists(input_path):
            if file_exists_in_dropbox(dropbox_output_path):
                print(f"✔️ Already uploaded: {dropbox_output_path}")
                success_count += 1
                continue

            success = trim_video(input_path, local_output_path)
            if success:
                upload_to_dropbox(local_output_path, dropbox_output_path)
                success_count += 1
            else:
                skipped_empty_count += 1
        else:
            missing_count += 1

    print(f"\nSplit: {split}")
    print(f"  ✅ Trimmed and uploaded: {success_count}")
    print(f"  ⚠️ Skipped (no active signing): {skipped_empty_count}")
    print(f"  ❌ Missing videos: {missing_count}")



Processing test split...


  0%|          | 1/500 [00:00<03:24,  2.44it/s]

✔️ Already uploaded: /ASL_trimmed/test/8126542305240905-PEOPLE.mp4


  0%|          | 2/500 [00:00<02:41,  3.09it/s]

✔️ Already uploaded: /ASL_trimmed/test/3129942110486452-THEN.mp4


...

✔️ Already uploaded: /ASL_trimmed/test/14992936899376108-LIKE.mp4


 90%|█████████ | 451/500 [08:31<03:17,  4.04s/it]

✔️ Already uploaded: /ASL_trimmed/test/39734251970462897-YOU.mp4


100%|██████████| 500/500 [12:47<00:00,  1.53s/it]


Split: test
  ✅ Trimmed and uploaded: 498
  ⚠️ Skipped (no active signing): 2
  ❌ Missing videos: 0


# Checking the Consistency of the trimmed videos
By length across each class

In [ ]:
# STEP 2: Install Dropbox SDK
!pip install dropbox

In [ ]:
import os
import cv2
import pandas as pd
from tqdm import tqdm
import dropbox
from tempfile import NamedTemporaryFile

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Dropbox configuration
DROPBOX_ACCESS_TOKEN = "sl.u.AFvPOA9BsaxGz8na17DzNOq-BOmm6cJjus70Ktx5Mph2CDZMJY_ehy0pzRapWt8UHdofn3Dg1SIhfRNbHcggKGYOPIBWWAB2b_8c-yRGCUEIVKJTJoQyy_NdKTJ_PXCgg7ruGaYC2YpzGmUWZ27RB_sny5YYHwTR1i2o4RIIlP3xNU2c0P3H8Z3bDl5xlDuTZJ5O6Q8ZT3dtJFNPlkUZxlO8p-fR5euXGyaA_iNwJfS8eD6K1tFu9yEZzDzti6XukTnHQLu9iV_MztYicbGr9r1GiAYuL7nAGi9FTb02I_V6IV8GrPC4f85QJ_u87ltKOITprGUheztqPP-leIv3ZHaunVQYz5tcCrFD08RJ9-Gtu7e8xgcTBDw6bbH3d27XyDqPgjrcdG96iOFGLCx7rLkrT5NQsQQDfbCal-QAcRQdccuAQ3Wk3ALnaEHVJP0Mb3R0DcemXWDyvp4NhtoGvh-IyBHgWkHJAN3gaMWfPyUtn29rARYkbVXHpXl8xAHVnO2zfCIjlA0Uyv4NrpJ_40VPHErR24hlN1VK9d6eVELOXDTU-usEqpF1YTJPi_iSUuXfnRPal3DTx9FfsoTvKfo69s0LlDOWcTvD7JZzY60pKLcvmmZkCmwYrAHl_mwMbLGKt-xM5rnGUQTsfEfdpWgeKoIvMbUz797QBR2RZVhyqFNCGdq-dz2c4ahkkrSvDR46yfky6yYDX6-zQCcAdwa0Owv7G8s2Aj2sgfuuYF5POilXvVZah7645aGyN65mLir1rJSgY2ChDaD9O8uBRFsqIHFHO189O8X6xWrgMLklcQxYY7yS36mVpwmHQ4wLCFANHdVexgkJC3OC4okMn2kq3QYnNBOrieDQaK88vc2zY0LPoNsPJPxuqKA0IYtq9wCbBwNDYf8xj0uRm-EHdIK4LZlracE7ObTMvF67fL4eruLKykO8u3NINEgoAHUpfxQXyHiOczFKDjyrCdBfCjqymzbymuSASae04XqiDlfs-xbgnd_sKCT3SXRblP8jcRRqqtJxcfYRz-jpau0S570TpSbhlDL-y6DA6nOsuMzcNHdOxkCr6emjoDYbXULDXd2xIb9TR2FPzlPzmv9oHEwltOL9BQv5R9BZdLEacDYn0EJ_jVOLF_hp9fRfhGWZKt9wxvZm5tXEWTo4GAsduiLShAtG1AriqEAl_CFswR34YjDWuALlqA1ZELXIXJT0CKkgtZk6ZP2uIzsoC-RrISDfHVj3n8UsKLZJ5CIQ8MXpoDHN-KzWSIOcXRzoIgyx0jVYoI4pY1yEUvFx08k-F9U8QepkyD9uXpeKZL452SChxCPRUP3VfZghT0RMSzBbT9M3QzhCJE2oixWrM1FzNSqw_NL7v_ub6Q5pIMcRhPTx3wyvzYRLMVjderir5CsIgqea0SFVWuO1jcHJ6FcooeyUEnhABWWRrv0YvqcPXieXxQ"
dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
dbx.users_get_current_account()

# Paths and splits
split_names = ['train', 'val', 'test']
split_dir = '/content/drive/MyDrive/ASL Project/splits'
dropbox_root = '/ASL_trimmed'

# Store all outliers
all_outliers = []

def get_video_frame_count_from_dropbox(dropbox_path):
    try:
        with NamedTemporaryFile(delete=False, suffix=".mp4") as tmp:
            metadata, res = dbx.files_download(dropbox_path)
            tmp.write(res.content)
            tmp_path = tmp.name

        cap = cv2.VideoCapture(tmp_path)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        os.remove(tmp_path)  # clean up
        return frames

    except Exception as e:
        print(f"❌ Error downloading or processing {dropbox_path}: {e}")
        return None

# Process each split
for split in split_names:
    print(f"\n🔍 Processing split: {split}")
    csv_path = os.path.join(split_dir, f"{split}_filtered.csv")
    if not os.path.exists(csv_path):
        print(f"❌ CSV not found: {csv_path}")
        continue

    df = pd.read_csv(csv_path)
    frame_data = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        video_file = row['Video file']
        gloss = row['Gloss'].strip().upper()
        dropbox_path = f"{dropbox_root}/{split}/{video_file}"

        frame_count = get_video_frame_count_from_dropbox(dropbox_path)
        if frame_count is not None:
            frame_data.append((split, gloss, video_file, frame_count))

    df_lengths = pd.DataFrame(frame_data, columns=['Split', 'Gloss', 'Video file', 'Frames'])
    gloss_stats = df_lengths.groupby('Gloss')['Frames'].agg(['mean', 'std']).reset_index()
    merged = pd.merge(df_lengths, gloss_stats, on='Gloss')
    merged['Z-Score'] = (merged['Frames'] - merged['mean']) / merged['std']
    merged['Outlier'] = merged['Z-Score'].abs() > 2

    all_outliers.append(merged[merged['Outlier']])

# Combine and display
outlier_df = pd.concat(all_outliers, ignore_index=True)
print(f"\n🚨 Found {len(outlier_df)} outlier videos across all splits")
from IPython.display import display
display(outlier_df[['Split', 'Gloss', 'Video file', 'Frames', 'Z-Score']])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🔍 Processing split: train


 40%|███▉      | 806/2031 [12:05<14:44,  1.38it/s]

❌ Error downloading or processing /ASL_trimmed/train/12041907153156517-DOG 3.mp4: ApiError('258fb7bd3edc4dcc9f25b31313fb2959', DownloadError('path', LookupError('not_found', None)))


 41%|████      | 825/2031 [12:21<14:40,  1.37it/s]

❌ Error downloading or processing /ASL_trimmed/train/22877680820569246-AND.mp4: ApiError('4dca4d1cad2546cf9bc0fd284e2b5147', DownloadError('path', LookupError('not_found', None)))


...

from matplotlib import pyplot as plt
_df_5['Z-Score'].plot(kind='line', figsize=(8, 4), title='Z-Score')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_6['Split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_6, x='Frames', y='Split', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['Split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='Z-Score', y='Split', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
display(outlier_df[['Split', 'Gloss', 'Video file', 'Frames', 'Z-Score']])


,Split,Gloss,Video file,Frames,Z-Score
0,train,APPLE,654679171689897-APPLE.mp4,91,3.022628
1,train,WE,5346957049353451-WE.mp4,121,2.969900
2,train,FRIEND,3845231217942484-FRIEND.mp4,79,2.128195
3,train,CAR,27759670594914465-CAR.mp4,104,2.178233
4,train,BUT,8335625687320589-BUT.mp4,86,2.259766
...,...,...,...,...,...
112,train,BACK,07598746238490839-seedBACK.mp4,102,3.220616
113,train,AT,6279791767383083-seedAT.mp4,95,2.559841
114,train,TAKE,8143899247512181-seedTAKE.mp4,77,2.500858
115,test,AND,3956799216747606-AND.mp4,122,2.016416


In [ ]:
import os
import dropbox
from pathlib import Path

# === CONFIG ===
DROPBOX_ACCESS_TOKEN = "sl.u.AFvPOA9BsaxGz8na17DzNOq-BOmm6cJjus70Ktx5Mph2CDZMJY_ehy0pzRapWt8UHdofn3Dg1SIhfRNbHcggKGYOPIBWWAB2b_8c-yRGCUEIVKJTJoQyy_NdKTJ_PXCgg7ruGaYC2YpzGmUWZ27RB_sny5YYHwTR1i2o4RIIlP3xNU2c0P3H8Z3bDl5xlDuTZJ5O6Q8ZT3dtJFNPlkUZxlO8p-fR5euXGyaA_iNwJfS8eD6K1tFu9yEZzDzti6XukTnHQLu9iV_MztYicbGr9r1GiAYuL7nAGi9FTb02I_V6IV8GrPC4f85QJ_u87ltKOITprGUheztqPP-leIv3ZHaunVQYz5tcCrFD08RJ9-Gtu7e8xgcTBDw6bbH3d27XyDqPgjrcdG96iOFGLCx7rLkrT5NQsQQDfbCal-QAcRQdccuAQ3Wk3ALnaEHVJP0Mb3R0DcemXWDyvp4NhtoGvh-IyBHgWkHJAN3gaMWfPyUtn29rARYkbVXHpXl8xAHVnO2zfCIjlA0Uyv4NrpJ_40VPHErR24hlN1VK9d6eVELOXDTU-usEqpF1YTJPi_iSUuXfnRPal3DTx9FfsoTvKfo69s0LlDOWcTvD7JZzY60pKLcvmmZkCmwYrAHl_mwMbLGKt-xM5rnGUQTsfEfdpWgeKoIvMbUz797QBR2RZVhyqFNCGdq-dz2c4ahkkrSvDR46yfky6yYDX6-zQCcAdwa0Owv7G8s2Aj2sgfuuYF5POilXvVZah7645aGyN65mLir1rJSgY2ChDaD9O8uBRFsqIHFHO189O8X6xWrgMLklcQxYY7yS36mVpwmHQ4wLCFANHdVexgkJC3OC4okMn2kq3QYnNBOrieDQaK88vc2zY0LPoNsPJPxuqKA0IYtq9wCbBwNDYf8xj0uRm-EHdIK4LZlracE7ObTMvF67fL4eruLKykO8u3NINEgoAHUpfxQXyHiOczFKDjyrCdBfCjqymzbymuSASae04XqiDlfs-xbgnd_sKCT3SXRblP8jcRRqqtJxcfYRz-jpau0S570TpSbhlDL-y6DA6nOsuMzcNHdOxkCr6emjoDYbXULDXd2xIb9TR2FPzlPzmv9oHEwltOL9BQv5R9BZdLEacDYn0EJ_jVOLF_hp9fRfhGWZKt9wxvZm5tXEWTo4GAsduiLShAtG1AriqEAl_CFswR34YjDWuALlqA1ZELXIXJT0CKkgtZk6ZP2uIzsoC-RrISDfHVj3n8UsKLZJ5CIQ8MXpoDHN-KzWSIOcXRzoIgyx0jVYoI4pY1yEUvFx08k-F9U8QepkyD9uXpeKZL452SChxCPRUP3VfZghT0RMSzBbT9M3QzhCJE2oixWrM1FzNSqw_NL7v_ub6Q5pIMcRhPTx3wyvzYRLMVjderir5CsIgqea0SFVWuO1jcHJ6FcooeyUEnhABWWRrv0YvqcPXieXxQ"  # Replace with your valid token
DROPBOX_ROOT = "/ASL_trimmed"  # Your Dropbox root folder structure
SAVE_BASE_DIR = "/content/drive/MyDrive/ASL Project/outlier_videos"

# === SETUP ===
dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
dbx.users_get_current_account()
Path(SAVE_BASE_DIR).mkdir(parents=True, exist_ok=True)

# === HELPER ===
def save_from_dropbox(dropbox_path, save_path):
    try:
        metadata, res = dbx.files_download(dropbox_path)
        with open(save_path, "wb") as f:
            f.write(res.content)
        print(f"✅ Saved: {save_path}")
    except Exception as e:
        print(f"❌ Failed to save {dropbox_path}: {e}")

# === SAVE VIDEOS FROM outlier_df ===
for _, row in outlier_df.iterrows():
    split = row['Split']
    video_file = row['Video file']

    dropbox_path = f"{DROPBOX_ROOT}/{split}/{video_file}"
    save_dir = os.path.join(SAVE_BASE_DIR, split)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, video_file)

    save_from_dropbox(dropbox_path, save_path)


✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/654679171689897-APPLE.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/5346957049353451-WE.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/3845231217942484-FRIEND.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/27759670594914465-CAR.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/8335625687320589-BUT.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/6776054154751192-CUT 1.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/014399429252126783-OVER.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/4665161320096789-THERE.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/8533448692981274-CAN 2.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/5395009645630173-BECAUSE.mp4
✅ Saved: /content/drive/MyDrive/ASL Project/outlier_videos/train/3162310724287807-H